In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
# Training Data Preprocessing
# Load data
train_data = pd.read_csv("train.csv")

# Separate features (X) and target (Y)
Y = pd.DataFrame(train_data["Calories"])
X = train_data.drop(columns=["Calories"])

# Apply a mapping to the 'Sex' column
X['Sex'] = X['Sex'].map({'male': 1, 'female': 0})

# Intialize data standradizer
scaler = StandardScaler()

# Standardize the data
X = scaler.fit_transform(X)

In [ ]:
# Testing Data Preprocessing
# Load data
X_test = pd.read_csv("test.csv")

# Apply a mapping to the 'Sex' column
X_test['Sex'] = X_test['Sex'].map({'male': 1, 'female': 0})

# Standradize the data
X_test = scaler.fit_transform(X_test)

y_test = (pd.read_csv("/kaggle/input/f-1-racer-diet-planning/sample_submission.csv")).drop(columns=['id'])

In [ ]:
# Simple Linear regression
from sklearn.linear_model import ElasticNet

# Create a linear regression model with standardization 
elastic_net = ElasticNet(alpha=0.2, l1_ratio=0.25)

# Train the model
elastic_net.fit(X, Y)

In [ ]:
# Make predictions on the test set using linear regression
linear_reg_predictions = elastic_net.predict(X_test)
print(linear_reg_predictions)

# Evaluate the model's performance
rmse_linear = np.sqrt(mean_squared_error(y_test, linear_reg_predictions))
print(f"Linear Regression RMSE: {rmse_linear:.2f}")

In [ ]:
from sklearn.svm import LinearSVC

# Create an instance of the Support Vector Regressor
svm_model = LinearSVC(penalty='l2', C=1.0, dual=False, random_state=42)

# Train the model
svm_model.fit(X, Y.values.ravel())

In [ ]:
# Make predictions
svm_predictions = svm_model.predict(X_test)

# Evaluate the model's performance
rmse_svm = np.sqrt(mean_squared_error(y_test, svm_predictions))
print(f"SVR with Linear Kernel RMSE: {rmse_svm:.2f}")

In [ ]:
from sklearn.ensemble import RandomForestRegressor, VotingRegressor

# Create an instance of the Random Forest Regressor
random_forest_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
random_forest_model.fit(X, Y.values.ravel())

In [ ]:
# Make predictions
rf_predictions = random_forest_model.predict(X_test)

# Evaluate the model's performance
rmse_rf = np.sqrt(mean_squared_error(y_test, rf_predictions))
print(f"Random Forest Regression RMSE: {rmse_rf:.2f}")

In [ ]:
ensemble_estimators = [
    ('svm', svm_model),
    ('RF', random_forest_model),
    ('LR', elastic_net)
]

# Create the voting model
ensemble_model = VotingRegressor(estimators=ensemble_estimators)
ensemble_model.fit(X_train_scaled, y_train)

In [ ]:
# Test the ensemble of models
predictions = ensemble_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print(f"Ensemble Model RMSE: {rmse:.4f}")